In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.pylab as pl
from matplotlib.colors import ListedColormap
%matplotlib inline

actions = [0,1,2,3]
action_coordinates = {0: np.array([1,0]),# down
                      1: np.array([0,1]), # right
                      2: np.array([-1,0]), # up
                      3: np.array([0,-1])} # left
gamma = .9

On considère un environnement où on se déplace sur une grille de taille 4 x 12. Un état est un couple de la forme $(i,j)$ où $0\leqslant i\leqslant 3$ et $0\leqslant j\leqslant 11$. Comme pour le labyrinthe, l'ensemble des actions est $\left\{ 0,1,2,3 \right\}$ qui correspondent respectivement à bas, droite, haut, gauche. Pour certains états et certaines actions, l'état suivant obtenu n'est pas celui auquel on s'attend. Le code suivant définit une class `env` qui simule les interactions avec l'environnement. On peut interagir avec l'environnement en utilisant une instance de cette classe, avec un code comme le suivant. 

In [ ]:
class env:
    height = 4
    width = 12
    s = (0,0)

    def __init__(self):
        return None

    def take_action(self, a):
        new_s = tuple(np.array(self.s)+action_coordinates[a])

        if new_s[0]<0 or new_s[1]<0 or new_s[0] >= self.height or new_s[1] >= self.width:
            r = 0
        elif new_s == (0,self.width-1):
            r = 1
            self.s = (0,0)
        elif new_s == (0,0):
            r = 0
        elif new_s[0]==0:
            r = -10 
            self.s = (0,0)
        else:
            r = 0
            self.s = new_s

        return r, self.s

environment = env() # initialisé à l'état (0,0)
r, s = env.take_action(1) # on choisit l'action 1 (droite), cela renvoie le gain r et le nouvel état s
print(s)

Le code suivant permettra de visualiser le chemin emprunté par une politique gloutonne par rapport à une fonction action-valeur.

In [ ]:
def plot_path(q):
    cmap = pl.cm.Reds
    my_cmap = cmap(np.arange(cmap.N))
    my_cmap[:,-1] = np.linspace(0, 1, cmap.N)
    my_cmap = ListedColormap(my_cmap)

    array = np.zeros_like(q[:,:,0])

    env_ = env()
    s = (0,0)
    prev_states = []

    while s not in prev_states:
        prev_states.append(s)
        array[s] = 1
        a = np.argmax(q[s])
        _, s = env_.take_action(a)

    fig, ax = plt.subplots(figsize=(q.shape[1]+1, q.shape[0]+1))
    ax.set_xticks(np.arange(0, q.shape[1]+1, 1), minor=True)
    ax.set_yticks(np.arange(0, q.shape[0]+1, 1), minor=True)
    ax.grid(which="minor", color='black', linestyle='-', linewidth=1)
    ax.set_xticks([])
    ax.set_yticks([])
    im = ax.imshow(array, cmap=my_cmap, interpolation='nearest', extent=[0, q.shape[1], 0, q.shape[0]], alpha=.5)
    plt.show()

Une fonction action-valeur sera représentée par un array de taille 4*12*4, où les deux premières dimensions correspondent à l'état et la dernière à l'action. 

*Question 1*: Écrire une fonction `draw_action_greedy_policy` qui renvoie l'action choisie (aléatoirement) par une politique $\varepsilon$-gloutonne.

In [ ]:
def draw_action_greedy_policy(s, q, eps=0):
    # On pourra utiliser les fonction np.where(), np.random.binomial(), np.random.randint()

    # Compléter

Pour implémenter les méthodes d'apprentissage par renforcement, il sera pratique de représenter les fonctions action-valeur de la façon suivante: d'une part pour chaque couple état-action, la somme des valeurs calculées pour les mises à jour (et non la moyenne), et d'autre part le nombre de mises à jour, de sorte que la fonction action-valeur qui est l'approximation de $q_*$ peut être déduite en divisant chaque composante par le nombre de mises à jour correspondant. La fonction suivante effectue ce calcul.

In [ ]:
def q_mean(q_cumul, n_updates):
    return np.divide(q_cumul, n_updates, out=np.zeros_like(q_cumul), where=(n_updates != 0))

*Question 2*: Implémenter le Q-learning, où étant donné un état $s$, l'action $a$ est choisie par une politique $\varepsilon$-gloutonne par rapport à la fonction état-valeur courante. Essayer différentes valeurs de $\varepsilon$.

In [ ]:
eps = .5
n_iter = 100000
environment = env()
s = (0,0)
q_cumul = np.zeros((env.height, env.width, 4))
n_updates = np.zeros((env.height, env.width, 4), dtype=int)
for k in range(n_iter):
    # compléter
plot_path(q_mean(q_cumul, n_updates))

*Question 3*: Implémenter SARSA. Comparer les résultats obtenus avec ceux issus du Q-learning. Quel est ici l'effet de la valeur de $\varepsilon$ ? Essayer également en faisant tendre $\varepsilon$ vers 0.